## Import modules, save data to Pandas dataframe

In [1]:
from __future__ import division, print_function

from modules import PerformParser as parser
import numpy as np
import pandas as pd

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import os
np.set_printoptions(precision=2)

In [2]:
init_notebook_mode(connected=True)

# Lets look at the variables stored in our dataframe

In [3]:
# read dict file, save as pandas DF
#datafile = parser.dictFileToDataFrame("data/2017-4-14-18-36/exp_data-2017-4-14-18-36.dict")
#datafile = parser.dictFileToDataFrame("data/2016-4-19-14-4/exp_data-2016-4-19-14-4.dict")
datafile = parser.dictFileToDataFrame("data/2016-4-29-11-56/exp_data-2016-4-29-11-56.dict")

#datafile = parser.readPerformDict("data/2016-4-29-11-56/exp_data-2016-4-29-11-56.dict")


# the frame of data that we will explore.  Arbitrary
frameIndex = 100

In [4]:
list(datafile.columns)

['IOD',
 'IPD',
 'ballFinalPos_XYZ',
 'ballInitialPos_XYZ',
 'ballInitialVel_XYZ',
 'ballPos_XYZ',
 'ballTTC',
 'ballVel_XYZ',
 'blankDur',
 'blockNumber',
 'calibrationCounter',
 'calibrationPos_XYZ',
 'cycEyeBasePoint_XYZ',
 'cycEyeInHead_XYZ',
 'cycEyeNodeInWorld_XYZ',
 'cycEyeOnScreen_XY',
 'cycGazeNodeInWorld_XYZ',
 'cycInverseMat_4x4',
 'cycMat_4x4',
 'eventFlag',
 'eyeTimeStamp',
 'frameTime',
 'inCalibrationQ',
 'isBallVisibleQ',
 'isCalibratedSMIQ',
 'leftEyeBasePoint_XYZ',
 'leftEyeInHead_XYZ',
 'leftEyeInverseMat_4x4',
 'leftEyeLensDistance',
 'leftEyeMat_4x4',
 'leftEyeNodeInHead_XYZ',
 'leftEyeNodeInWorld_XYZ',
 'leftEyeOnScreen_XY',
 'leftEyeScreenDistance',
 'leftGazeNodeInWorld_XYZ',
 'leftPupilPos_XYZ',
 'leftPupilRadius',
 'paddleMat_4x4',
 'paddlePos_XYZ',
 'paddleQuat_XYZW',
 'postBlankDur',
 'preBlankDur',
 'rightEyeBasePoint_XYZ',
 'rightEyeInHead_XYZ',
 'rightEyeInverseMat_4x4',
 'rightEyeLensDistance',
 'rightEyeMat_4x4',
 'rightEyeNodeInHead_XYZ',
 'rightEyeNod

# Before we can plot any data, we have to make sure we understand the related coordinate systems.

<img src="graphics/coordSys.png",width=1000>

Raw eye-in-head data (below) indicates the direction of gaze within SMI's head-centered coordinate system (left figure).  As you would expect from the figure above, gaze is primarily directed up +Z.

In [5]:
datafile['cycEyeInHead_XYZ'][0:10]

0    [0.129151, -0.309052, 0.942235]
1    [0.128799, -0.306807, 0.943017]
2    [0.128799, -0.306807, 0.943017]
3    [0.129543, -0.309401, 0.942067]
4    [0.130465, -0.308183, 0.942339]
5    [0.131372, -0.308018, 0.942267]
6    [0.129326, -0.310528, 0.941726]
7    [0.129326, -0.310528, 0.941726]
8    [0.128504, -0.310706, 0.941779]
9    [0.129196, -0.311404, 0.941454]
Name: cycEyeInHead_XYZ, dtype: object

### Because the rest of our data is stored using the Vizard coordinate system, the first thing we should do is convert the SMI data to be consistent with our Vizard conventions. 

Notice that the the X axis is flipped between SMI and Vizard.  We will have to account for that.
Flip the X axis for all the cyclopean gaze data by applying an anonymous function to each row of the dataframe.

In [6]:
datafile['cycEyeInHead_XYZ'] = datafile['cycEyeInHead_XYZ'].apply(lambda row: [-row[0],row[1],row[2]])

## Plot it!
I use the Plotly module to plot the data. 

### Note that Vizard and Plotly swap Y and Z axes (see coordinate systems above)
I handle that upon plotting. If the data xyz = [x,y,z], then I plot x=xyz[0], x=xyz[2], and x=xyz[1]

In [7]:
from plotly.graph_objs import *

# Plot for a single frame, 10.
frameIdx = 100;
            
cycEyePos = Scatter3d(x=[0],
                   y=[0],
                   z=[0],
                   mode='markers',
                      marker={'color': '#48186a', 'size': 10},
                     )

cycEyeInHead_XYZW = datafile['cycEyeInHead_XYZ'][frameIdx]

eihDir = Scatter3d(x=[0,cycEyeInHead_XYZW[0]],
                   y=[0,cycEyeInHead_XYZW[2]],
                   z=[0,cycEyeInHead_XYZW[1]],
                   mode='lines',
                   line = dict(
                       color = ('rgb(205, 12, 24)'),
                       width = 4)
                  )

layout = Layout(title="EIH", 
                width=800,
                height=800,
                showlegend=False,
                scene=Scene(aspectmode='manual',
                            aspectratio=dict(x=1, y=1, z=1),
                            xaxis=dict(range=[-1, 1], title='x Axis'),
                            yaxis=dict(range=[-1, 1], title='y Axis'),
                            zaxis=dict(range=[-1, 1], title='z Axis'),

                           ),
                margin=Margin(t=100),
                hovermode='closest',
                
                )

data=Data([cycEyePos,eihDir])

fig=Figure(data=data,layout=layout)

plot(fig)

'file:///Users/gjdiaz/Documents/Code/Python/GazeToolbox/ECEM-2017-Eyetracking-in-VR/temp-plot.html'

# Validate

The figure above is interactive!  Try clicking on the time series below to quickly jump about the timeseries.  You can also adjust the window size to zoom in at different temporal scales.

# Lets calculate velocity of the cyclopean vector!

I have to do a bit of magic to convert the eye tracker time sample into ∆t values

In [8]:
datafile['smiDateTime'] = pd.to_datetime(datafile.eyeTimeStamp,unit='ns')
deltaTime = datafile['smiDateTime'].diff()
deltaTime.loc[deltaTime.dt.microseconds==0] = pd.NaT
deltaTime = deltaTime.fillna(method='bfill', limit=1)
datafile['smiDeltaT'] = deltaTime.dt.microseconds / 1000000

Take the angular difference:  $arccos(eih_{t}•eih_{t-1}))$.
### You will get a warning when...
$eih_{t} = eih_{t-1}$, which is the case when the eye tracking data did not change across subsequent samples.  This will happen sometimes, as the sampling rate for the SMI eye tracker was 60 Hz, and the display updates at 70 Hz. In this case, the function will return NaN.

In [9]:
changeInAngle_fr = [ np.rad2deg(np.arccos(np.vdot(abc,xyz)))
                    for abc, xyz in zip(datafile['cycEyeInHead_XYZ'].values, np.roll(datafile['cycEyeInHead_XYZ'],1))]

/Users/gjdiaz/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning:

invalid value encountered in arccos



### Divide by time to get velocity

In [10]:
datafile['cycGazeVelocity'] = changeInAngle_fr / datafile['smiDeltaT']

Fill in nan values with the previous velocity value.

In [11]:
datafile['cycGazeVelocity'] = datafile['cycGazeVelocity'].fillna(method='bfill')

In [17]:
datafile['cycGazeVelocity'][1:10]

1     7.730643
2     9.019481
3     9.019481
4     3.673823
5    10.460367
6    10.460367
7     3.120626
8     3.120626
9     5.065125
Name: cycGazeVelocity, dtype: float64

### Plot!
I have provided a simple plotting function.  If you want to look at it, have a look at the file: /modules/ecem2017.py

You will notice that the figure has event labels at the time of important events during the trial.

<img src="graphics/eventLabels.png",width=500>

In [18]:
from modules.ecem2017 import *

In [19]:
plotGazeVelocity(datafile, 
                  trialNumber=20, 
                  columnNames = ['cycGazeVelocity'],
                  width=1000,
                  height=600,
                  yLim=[0,500])